# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
\mu = \int r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis, tol=tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770066784767                   -0.53    8.0
  2   -2.771673489198       -2.79       -1.29    1.0
  3   -2.771714629059       -4.39       -2.80    2.0
  4   -2.771714714434       -7.07       -4.08    2.0
  5   -2.771714715241       -9.09       -4.88    2.0


-0.00013254342476113

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε, tol=tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770328367590                   -0.53    8.0
  2   -2.771768726032       -2.84       -1.30    1.0
  3   -2.771801257650       -4.49       -2.64    1.0
  4   -2.771802072023       -6.09       -3.97    2.0
  5   -2.771802074381       -8.63       -4.55    2.0


0.017618326137534442

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013254342476113
Displaced dipole:  0.017618326137534442
Polarizability :   1.7750869562295573


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $\chi_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $\delta V_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
\delta\rho = \chi_0 \delta V = \chi_0 (\delta V_{\rm ext} + K \delta\rho),
$$
which implies
$$
\delta\rho = (1-\chi_0 K)^{-1} \chi_0 \delta V_{\rm ext}.
$$
From this we identify the polarizability operator to be $\chi = (1-\chi_0 K)^{-1} \chi_0$.
Numerically, we apply $\chi$ to $\delta V = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply (1- χ0 K)
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; ρ=res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# δVext is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply χ0 once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493932399248e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.765606978181e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.828389331943e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694639816190e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.096300668717e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 2.694602667557e-10
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.090604382695e-11
[ Info: GMRES linsolve in iter 1; step 8: normres = 1.123769095073e-13
[ Info: GMRES linsolve in iter 1; finished at step 8: normres = 1.123769095073e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.654626855751e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 8.172367276067e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 5.970237643671e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.608204424761e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.